# MGL869 - Projet personnel

*MGL869 ETS Montreal - Production engineering*

## Abstract

## Authors
- **William PHAN**

---

## Part 1 : Collecte des données

In [1]:
from Jira import jira_download
from pandas import Index
from numpy import ndarray


### 1.1 - Téléchargement des données Jira

Nous téléchargeons les données si elles ne sont pas déjà présentes dans le dossier de données.

Renvoie le dataframe des données.

Le filtre de requête peut être défini dans le fichier config.ini.

In [2]:
jira_dataframe = jira_download()

Data already downloaded
Filter = 'project=HIVE AND issuetype=Bug AND status in (Resolved, Closed) AND affectedVersion>= 2.0.0'


### 1.2 - Nettoyer les données Jira en utilisant pandas

Auparavant, nous avons téléchargé toutes les données de Jira. Maintenant, nous allons nettoyer les données en utilisant pandas. Nous allons conserver seulement certaines colonnes et combiner certaines colonnes.

In [3]:
keep: [str] = ['Issue key', 'Status', 'Resolution', 'Created', 'Fix Versions Combined', 'Affects Versions Combined']

In [4]:
affects_version_columns: [str] = [col for col in jira_dataframe.columns if col.startswith('Affects Version/s')]
jira_dataframe['Affects Versions Combined'] = jira_dataframe[affects_version_columns].apply(
    lambda x: ', '.join(x.dropna().astype(str)), axis=1
)

In [5]:
# Combine the versions into a single column
fix_version_columns: [str] = [col for col in jira_dataframe.columns if col.startswith('Fix Version/s')]

jira_dataframe['Fix Versions Combined'] = jira_dataframe[fix_version_columns].apply(
    lambda x: ', '.join(x.dropna().astype(str)), axis=1
)
jira_dataframe = jira_dataframe.loc[:, keep]

In [6]:
# Identify columns whose names contain the string 'Issue key'
issue_key_columns: Index = jira_dataframe.columns[jira_dataframe.columns.str.contains('Issue key')]
# Extract the values from these columns as a NumPy array
issue_key_values: ndarray = jira_dataframe[issue_key_columns].values
# Flatten the array to create a one-dimensional list of all 'Issue key' values
flattened_issue_keys: ndarray = issue_key_values.flatten()
# Convert the list into a set to remove duplicates
ids: set = set(flattened_issue_keys)

---


## Part 2 : Analyse du répo git


In [7]:
from Hive import git_download, commit_analysis, update_commit_dataframe, filter_versions_by_min
from git import Repo, Tag
from pandas import DataFrame
from configparser import ConfigParser
from re import compile
from packaging import version  

### 2.1 - Clonage du répo

In [8]:
repo: Repo = git_download()

Output/hive_data/hiveRepo False
Pulling the repository: https://github.com/apache/hive.git


In [9]:
all_couples = commit_analysis(ids)

20524 couples found.


### 2.2 - Filtrage des données et couples

In [10]:
commit_dataframe: DataFrame = DataFrame(all_couples, columns=["Issue key", "File", "Commit"])

In [11]:
# Languages without whitespaces
config: ConfigParser = ConfigParser()
config.read("config.ini")
languages: [str] = config["GENERAL"]["Languages"].split(",")
languages: [str] = [lang.strip() for lang in languages]
commit_dataframe: DataFrame = commit_dataframe[commit_dataframe['File'].str.endswith(tuple(languages))]

In [12]:
couples = update_commit_dataframe(commit_dataframe, jira_dataframe)
couples
# filtered_couples = couples[couples['Version Affected'].str.contains('2.3.9', na=False)]
# filtered_couples

,Issue key,File,Version Affected
0,HIVE-21614,ql/src/test/org/apache/hadoop/hive/metastore/T...,"2.3.4, 3.0.0"
1,HIVE-21614,standalone-metastore/metastore-server/src/main...,"2.3.4, 3.0.0"
2,HIVE-21614,standalone-metastore/metastore-server/src/main...,"2.3.4, 3.0.0"
3,HIVE-28366,iceberg/iceberg-handler/src/main/java/org/apac...,4.0.0
4,HIVE-28366,iceberg/iceberg-handler/src/main/java/org/apac...,4.0.0
...,...,...,...
10268,HIVE-17720,ql/src/java/org/apache/hadoop/hive/ql/metadata...,3.0.0
10269,HIVE-17706,itests/util/src/main/java/org/apache/hadoop/hi...,3.0.0
10270,HIVE-17706,itests/util/src/main/java/org/apache/hive/beel...,3.0.0
10271,HIVE-17706,itests/util/src/main/java/org/apache/hive/beel...,3.0.0


### 2.3 - Collecte des versions filtrées

In [13]:
releases_regex: [str] = config["GIT"]["ReleasesRegex"].split(",")
tags: Tag = repo.tags
versions: dict = {tag.name: tag.commit for tag in tags}
releases_regex: [str] = [regex.strip() for regex in releases_regex]
releases_regex = [compile(regex) for regex in releases_regex]

## Part 3. - Dynamic Metrics

In [14]:
from Dynamic import convert_json_to_csv, merge_static_and_dynamic_csv, build_dependencies, display_hierarchy, collect_dynamic_metrics_v2
from Hive import filter_versions_by_min

In [15]:
all_versions = filter_versions_by_min(versions, releases_regex,'1.0')
version_json = build_dependencies(all_versions)
#display_hierarchy(version_json)
#version_json

In [16]:
dynamic_metrics = collect_dynamic_metrics_v2(version_json)

Dynamic Metrics have already been collected. Skipping...


In [17]:
convert_json_to_csv()

Conversion of dynamic metrics to csv has already been done. Skipping...


In [18]:
merge_static_and_dynamic_csv()

Merging has already been done. Skipping...
